In [2]:
%pip install transformers

Note: you may need to restart the kernel to use updated packages.


### Load and Pre-process Dataset

In [3]:
%pip install datasets

In [2]:
%pip install transformers

In [4]:
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install tf-keras

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 18.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [8]:
%pip install transformers[torch]

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer

# Load the IMDB dataset
dataset = load_dataset("imdb")

# Load a pre-trained tokenizer (BERT in this case)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Apply tokenizer to train and test datasets
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Set format for PyTorch
tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

### Fine-tune Model

In [2]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

# Load a pre-trained BERT model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",          # output directory
    num_train_epochs=3,              # number of training epochs
    per_device_train_batch_size=8,   # batch size for training
    per_device_eval_batch_size=16,   # batch size for evaluation
    evaluation_strategy="epoch",     # evaluation strategy to use
    logging_dir="./logs",            # directory for storing logs
    logging_steps=10,
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)

# Train the model
trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\mindf\anaconda3\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/9375 [00:00<?, ?it/s]

{'loss': 0.718, 'grad_norm': 2.3686017990112305, 'learning_rate': 4.994666666666667e-05, 'epoch': 0.0}
{'loss': 0.7487, 'grad_norm': 3.226097583770752, 'learning_rate': 4.989333333333334e-05, 'epoch': 0.01}
{'loss': 0.6694, 'grad_norm': 5.74545955657959, 'learning_rate': 4.9840000000000004e-05, 'epoch': 0.01}
{'loss': 0.6614, 'grad_norm': 2.566739559173584, 'learning_rate': 4.978666666666667e-05, 'epoch': 0.01}
{'loss': 0.535, 'grad_norm': 3.0185539722442627, 'learning_rate': 4.973333333333334e-05, 'epoch': 0.02}
{'loss': 0.5512, 'grad_norm': 8.104778289794922, 'learning_rate': 4.9680000000000005e-05, 'epoch': 0.02}
{'loss': 0.5818, 'grad_norm': 12.933828353881836, 'learning_rate': 4.962666666666667e-05, 'epoch': 0.02}
{'loss': 0.5108, 'grad_norm': 5.880299091339111, 'learning_rate': 4.9573333333333335e-05, 'epoch': 0.03}
{'loss': 0.3936, 'grad_norm': 12.548469543457031, 'learning_rate': 4.952e-05, 'epoch': 0.03}
{'loss': 0.5229, 'grad_norm': 11.821054458618164, 'learning_rate': 4.9466

KeyboardInterrupt: 

### Save Model

In [ ]:
# Save the fine-tuned model locally
model.save_pretrained("./finetuned_model")
tokenizer.save_pretrained("./finetuned_model")
